In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上进行端到端机器学习：MLOps阶段1：数据管理：开始使用Vertex AI数据集

在Colab中运行 Github中查看 在Vertex AI Workbench中打开

## 概述

本教程演示了如何在Google Cloud上生产环境中使用Vertex AI进行端到端MLOps。本教程涵盖了第一阶段：数据管理：开始使用Vertex数据集。

### 目标

在本教程中，您将学习如何使用“顶点 AI 数据集”来进行“顶点 AI”训练。

本教程使用以下 Google Cloud ML 服务：

-“顶点 AI 数据集”
-“BigQuery 数据集”

执行的步骤包括：

- 为以下类型创建一个顶点 AI“数据集”资源：
    - 图像数据
    - 文本数据
    - 视频数据
    - 表格数据
    - 预测数据

- 使用筛选器搜索“数据集”资源。
- 将“BigQuery”数据集的样本读入数据框中。
- 使用 TensorFlow 数据验证从数据框中的样本生成统计信息和数据架构。
- 使用 TensorFlow 数据验证检测新数据中的异常。
- 使用 TensorFlow Transform从数据架构生成一个TFRecord特征规范。
- 导出数据集并转换为TFRecords。

### 推荐

在谷歌云上进行端到端MLOps时，以下是使用Vertex AI 数据集的最佳实践：

- 对于图像数据，请使用CSV索引文件格式
- 对于文本数据，请使用CSV索引文件格式：
    - 对于短文本字符串，请将文本字符串嵌入到CSV文件中。
    - 对于长文本字符串，请将文本放在引用的文本文件中。


- 对于视频数据，请使用JSON索引文件格式
- 对于表格数据：
    - 对于小型数据集，请使用CSV索引文件格式。
    - 对于大型数据集，请使用BigQuery表格。


- 在`list()`方法中使用`filter`和`order_by`参数来查找数据集的最新版本。

- 当使用`Vertex AI 数据集`进行自定义训练时：
    - 对于表格数据：
        - 使用CSV索引文件或BigQuery表格的引用。
        - 从CSV索引文件/BigQuery表格创建一个tf.data.Dataset生成器。
    - 对于图像/视频数据：
        - 将数据导出为JSONL索引文件。
        - 使用索引文件，将图像/视频和标签转换为TFRecords。
        - 从TFRercords创建一个tf.data.Dataset生成器。
    - 对于文本数据：
        - 如果文本字符串被嵌入：
            - 转换为CSV文件。
            - 从CSV索引文件创建一个tf.data.Dataset生成器。
        - 如果文本字符串在文本文件中：
            - 使用JSON索引文件，将文本文件和标签转换为TFRecords。
            - 从TFRecords创建一个tf.data.Dataset。

数据集

本教程使用各种公共数据集来演示如何使用 `Vertex AI` 托管数据集。

费用
本教程使用谷歌云的可计费组件：
- Vertex AI
- 云存储
- BigQuery

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)、[云存储定价](https://cloud.google.com/storage/pricing)以及[BigQuery定价](https://cloud.google.com/bigquery/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装执行这个笔记本所需的包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install -U tensorflow $USER_FLAG -q
! pip3 install -U tensorflow-data-validation $USER_FLAG -q
! pip3 install -U tensorflow-transform $USER_FLAG -q
! pip3 install -U tensorflow-io $USER_FLAG -q
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-bigquery $USER_FLAG -q
! pip3 install -U tensorflow-io==0.18 $USER_FLAG -q
! pip3 install --upgrade db-dtypes $USER_FLAG -q! pip3 install --upgrade future $USER_FLAG -q

### 重新启动内核

安装完附加包后，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的 Google Cloud 项目

**无论您使用的是什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个帐户时，您将获得 $300 的免费信用用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目 ID。然后运行这个单元格，确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 运行以 `!` 为前缀的行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入到这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在进行直播教程会话，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源中发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench笔记本**，则您的环境已经经过验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过OAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到 [创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，并点击**创建**。

在**将此服务账户授权访问项目**部分，点击角色下拉列表。在筛选框中输入"Vertex"，并选择**Vertex管理员**。在筛选框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务账户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要完成以下步骤。**

当您使用 Vertex SDK 提交自定义训练作业时，您需要将包含训练代码的 Python 软件包上传到一个云存储桶中。Vertex AI 会从这个软件包中运行代码。在本教程中，Vertex AI 还会将训练作业生成的训练模型保存在同一个存储桶中。然后您可以基于这个输出创建一个 `Endpoint` 资源，以便提供在线预测。

请设置您的云存储桶的名称。存储桶名称在所有谷歌云项目范围内必须是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您尚未拥有该桶时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限：

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库和定义常量

将BigQuery包、TensorFlow数据验证(TFDV)包和TensorFlow数据验证包导入到你的Python环境中。

将TensorFlow转换(TFT)包和pandas导入到你的Python环境中。

In [ ]:
import google.cloud.aiplatform as aip
import pandas as pd
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from google.cloud import bigquery

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

创建 BigQuery 客户端。

In [ ]:
bqclient = bigquery.Client()

## Vertex AI数据集

Vertex AI `数据集` 是在Vertex AI服务中管理数据集的方式。Vertex AI数据集也被称为`数据集`资源。有四种类型的`数据集`资源，针对不同的数据类型：

- `ImageDataset`：图像数据
- `TabularDataset`：表格（结构化）数据
- `TextDataset`：文本（自然语言）数据
- `VideoDataset`：视频数据
- `TimeSeriesDataset`：预测数据

Vertex AI `数据集`提供以下功能：

- 用于自动（编程）过程的独特内部标识符。
- 为交互式过程指定的用户标识符（显示名称）。
- 兼容AutoML训练。
- 为自定义训练导出数据集。
- 数据集搜索功能。
- 创建/更新时间戳。
- 统计数据

了解更多关于[所有数据集文档](https://cloud.google.com/vertex-ai/docs/datasets/datasets)。

创建图像数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项目导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式：
  - `single_label`：二元和多类分类
  - `multi_label`：多标签多类分类
  - `bounding_box`：目标检测
  - `image_segmentation`：分割

了解有关[ImageDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)的更多信息。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

In [ ]:
dataset = aip.ImageDataset.create(
    display_name="example" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建视频数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式。
  - `classification`：二元和多类分类
  - `object_tracking`：对象跟踪
  - `action_recognition`：动作识别

了解有关[VideoDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)的更多信息。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

In [ ]:
dataset = aip.VideoDataset.create(
    display_name="example" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### 创建文本数据集

接下来，使用`create`方法为`TextDataset`类创建`Dataset`资源，需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标注模式。
  - `single_label`：二元和多类分类
  - `multi_label`：多标签多类分类
  - `sentiment`：情感分析
  - `extraction`：实体抽取

了解更多关于[TextDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

In [ ]:
dataset = aip.TextDataset.create(
    display_name="example" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### 创建一个表格数据集

#### CSV 输入数据

接下来，使用 CSV 输入数据创建 `Dataset` 资源，为了 `TabularDataset` 类使用 `create` 方法，方法接受以下参数:

- `display_name`: `Dataset` 资源的可读名称。
- `gcs_source`: 一个或多个数据集索引文件的列表，用于将数据项导入到 `Dataset` 资源中。

了解更多关于[从 CSV 文件创建 TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="example" + "_" + TIMESTAMP, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

#### BigQuery 输入数据

接下来，使用`TabularDataset`类的`create`方法为BigQuery表输入创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `bq_source`：要将数据项导入到`Dataset`资源中的一个或多个BigQuery表的列表。

了解更多关于[来自BigQuery表的TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-pythonn)

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="example" + "_" + TIMESTAMP, bq_source=[IMPORT_FILE]
)

print(dataset.resource_name)

#### 数据框输入数据

接下来，使用`create_from_dataframe`方法为pandas数据框输入创建`TabularDataset`类的`Dataset`资源，参数如下：

- `display_name`：`Dataset`资源的人类可读名称。
- `df_source`：要将数据项目导入至`Dataset`资源的pandas数据框。
- `staging_path`：用于存储导入数据的BigQuery表。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(BQ_TABLE)

rows = bqclient.list_rows(
    table,
    max_results=10000,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

In [ ]:
dataset = aip.TabularDataset.create_from_dataframe(
    display_name="example" + "_" + TIMESTAMP,
    df_source=dataframe,
    staging_path=f"bq://{PROJECT_ID}.samples.gsod",
)

print(dataset.resource_name)

创建一个时间序列数据集

接下来，使用`TimeSeriesDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `bq_source`：或者，从BigQuery表中导入数据项到`Dataset`资源。

了解更多关于[TimeSeriesDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-tabular)。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/ai-platform/covid/bigquery-public-covid-nyt-us-counties-train.csv"

In [ ]:
dataset = aip.TimeSeriesDataset.create(
    display_name="example" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
)

print(dataset.resource_name)

## Vertex AI数据集属性和方法

以下是`Dataset`的方法：

- `list()`: 列出`Dataset`资源的实例。
- `import_data()`: 将额外数据导入`Dataset`资源。
- `export_data()`: 为定制训练导出数据集索引文件。
- `delete()`: 删除数据集。
- `update()`: 尚未实现。

通过在Python中执行help(method_name)来获取每个方法的更多信息。

以下是`Dataset`的属性：

- `name`: 内部唯一标识符。
- `resource_name`: 完全合格的内部唯一标识符。
- `display_name`: 人员分配的标识符。
- `create_time`: 创建数据集时的时间戳。
- `update_time`: 上次更新数据集时的时间戳。
- `metadata_schema_uri`: 数据标记模式。

### 列出数据集

`list()`方法返回相应数据类型（例如，表格）的所有数据集，以列表形式呈现。

In [ ]:
datasets = aip.TabularDataset.list()
for _dataset in datasets:
    print(_dataset.name)

### 列出符合筛选条件的数据集

`list()` 方法支持仅返回符合`筛选条件`的数据集。例如，显示名称与指定显示名称匹配的所有数据集：
```
    list(filter='display_name=my_display_name')
```

当搜索匹配多个数据集时，可以根据数据集属性对列表进行排序。例如，按创建时间排序，第一个数据集为最新的：

```
    list(filter='display_name=my_display_name,order_by=create_time')
```

In [ ]:
datasets = aip.TabularDataset.list(
    filter=f'display_name="example_{TIMESTAMP}"', order_by="create_time"
)
latest_dataset = datasets[0]
print(latest_dataset)

TensorFlow数据验证

TensorFlow数据验证（TFDV）软件包与Vertex AI和BigQuery数据集一起使用，用于：

- 生成数据集统计信息。
- 为数据验证生成数据模式。
- 使用数据模式检测新数据中的异常。
- 生成特征规范，以便将数据转换为TFRecords。

了解更多关于[TensorFlow数据验证：入门](https://www.tensorflow.org/tfx/data_validation/get_started)。

### 将BigQuery数据集读入pandas dataframe

接下来，您可以使用BigQuery的`list_rows()`和`to_dataframe()`方法，将数据集的样本读入pandas dataframe，具体步骤如下：

- `list_rows()`: 在指定的表上执行查询，并返回查询结果的行迭代器。可以选择指定：
  - `selected_fields`: 要返回的字段（列）的子集。
  - `max_results`: 要返回的行数的最大值。与SQL LIMIT命令相同。

- `rows.to_dataframe()`: 调用行迭代器，并将数据读入pandas dataframe。

了解更多关于[将BigQuery表加载到dataframe中](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)。

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string("bigquery-public-data.samples.gsod")

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

### 生成数据集统计信息

#### 数据框输入数据

使用 TensorFlow 数据验证 (TFDV) 软件包对数据集生成统计信息。使用`generate_statistics_from_dataframe()`方法，并使用以下参数：

- `dataframe`：存储在内存中的 pandas 数据框中的数据集。
- `stats_options`：选择的统计选项：
  - `label_feature`：要预测的标签列。
  - `sample_rate`：抽样率。如果指定，则对样本进行统计。
  - `num_top_values`：要保留的最常见特征值的数量（对于字符串特征）。

了解有关 [TensorFlow 数据验证(TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started)的更多信息。

In [ ]:
stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataframe,
    stats_options=tfdv.StatsOptions(
        label_feature="mean_temp", sample_rate=1, num_top_values=50
    ),
)

print(stats)

### 可视化数据集统计信息

使用TFDV的`visualize_statistics()`方法可以显示数据集统计信息的可视化。

In [ ]:
tfdv.visualize_statistics(stats)

### 生成原始数据模式

使用TensorFlow数据验证（TFDV）包在数据集上生成数据模式。使用`infer_schema()`方法，具有以下参数：

- `statistics`：由TFDV生成的统计信息。

In [ ]:
schema = tfdv.infer_schema(statistics=stats)
print(schema)

### 在额外数据中检测异常值

当为数据集提供了额外数据时，您可以使用TFDV的`validate_statistics`方法检查新数据与旧数据之间的异常值。

以下代码示例通过获取下一批数据（第2批500行）并将额外字段添加到数据帧中，为新数据生成统计信息来模拟新数据。

`validate_statistics()`方法是用以下参数调用的：

- `statistics`：新数据的统计信息。
- `schema`：旧数据的数据模式。

In [ ]:
# Download a table.
table = bigquery.TableReference.from_string("bigquery-public-data.samples.gsod")

rows = bqclient.list_rows(
    table,
    max_results=500,
    start_index=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
        bigquery.SchemaField("num_mean_temp_samples", "INTEGER"),
    ],
)

dataframe = rows.to_dataframe()

new_stats = tfdv.generate_statistics_from_dataframe(
    dataframe=dataframe,
    stats_options=tfdv.StatsOptions(
        label_feature="mean_temp", sample_rate=1, num_top_values=50
    ),
)

anomalies = tfdv.validate_statistics(statistics=new_stats, schema=schema)
print(anomalies.anomaly_info)

生成特征规范，在带有TensorFlow Transform（TFT）包的数据集上与TFRecords兼容。使用`schema_as_feature_spec()`方法，使用以下参数：

- `schema`：由TFDV生成的数据模式。

In [ ]:
feature_spec = tft.tf_metadata.schema_utils.schema_as_feature_spec(schema).feature_spec

print(feature_spec)

### 将数据集复制到云存储

接下来，您可以使用BigQuery提取命令，将BigQuery数据集复制为CSV文件到云存储。

了解更多关于[BigQuery命令行界面](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference)。

In [ ]:
comps = BQ_TABLE.split(".")
BQ_PROJECT_DATASET_TABLE = comps[0] + ":" + comps[1] + "." + comps[2]

! bq --location=us extract --destination_format CSV $BQ_PROJECT_DATASET_TABLE $BUCKET_URI/mydata*.csv

IMPORT_FILES = ! gsutil ls $BUCKET_URI/mydata*.csv

print(IMPORT_FILES)

EXAMPLE_FILE = IMPORT_FILES[0]

! gsutil cat $EXAMPLE_FILE | head

### 生成数据集统计

#### CSV输入数据

使用TensorFlow数据验证（TFDV）包对数据集生成统计信息。使用`generate_statistics_from_csv()`方法，并设置以下参数：

- `data_location`：数据集在云存储中的文件位置。
- `stats_options`：选择的统计选项：
  - `label_feature`：要预测的列。
  - `num_top_values`：保留字符串特征中出现频率最高的特征值的数量。

了解有关[TensorFlow数据验证（TFDV）](https://www.tensorflow.org/tfx/data_validation/get_started)的信息。

In [ ]:
stats = tfdv.generate_statistics_from_csv(
    data_location=EXAMPLE_FILE,
    stats_options=tfdv.StatsOptions(label_feature="mean_temp", num_top_values=50),
)

print(stats)

### 将“文本数据集”导出为pandas数据帧。

属性`gca_resource.metadata['inputConfig']['gcsSource']['uri']`包含一个或多个导入的CSV文件的列表。

要从多个CSV源创建数据帧，您需要读取每个CSV文件并将数据帧连接在一起。

如果您在Colab上运行此笔记本，请运行以下单元格安装软件包fsspec和gcsfs。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Workbench AI Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        ! pip3 install fsspec
        ! pip3 install gcsfs

In [ ]:
all_files = dataset.gca_resource.metadata["inputConfig"]["gcsSource"]["uri"]
df = pd.concat(pd.read_csv(f) for f in all_files)

print(df.head)

导出数据集索引

接下来，您将数据集索引导出到一个 JSONL 文件中，该文件将被您自定义的训练作业使用，以获取用于训练模型的数据和相应的标签。通过调用 export_data() 方法生成 JSONL 索引文件，其中包括以下参数:
 - `output_dir`: 用于将 JSONL 数据集索引文件写入的云存储桶。

In [ ]:
datasets = aip.ImageDataset.list(
    filter=f'display_name="example_{TIMESTAMP}"', order_by="create_time"
)
latest_dataset = datasets[0]
print(latest_dataset)

In [ ]:
dataset = latest_dataset

In [ ]:
EXPORTED_DIR = f"{BUCKET_URI}/exported"
exported_files = dataset.export_data(output_dir=EXPORTED_DIR)

! gsutil ls $EXPORTED_DIR

快速查看您导出的数据集索引文件

快速查看导出的数据集索引文件的内容。当`export_data()`完成时，该方法返回一个路径列表，指向导出的数据集索引文件。

获取导出的数据集索引文件的路径（`exported_files[0]`），然后显示文件中的第一个十个JSON对象，即数据项。

每个数据项的JSONL格式为：

    { "imageGcsUri": 图像路径, "classificationAnnotation": { "displayName": 标签 } }

In [ ]:
jsonl_index = exported_files[0]

! gsutil cat $jsonl_index | head

#### 读取索引文件

您需要在自定义训练的Python脚本中添加代码，以读取导出的数据集索引，这样您就可以为自定义训练您的模型生成训练批次。

以下是您可能如何读取导出的数据集索引文件的示例：

1. 使用Tensorflow的Cloud Storage文件方法打开文件（`tf.io.gfile.GFile()`），并读取所有行（`f.readlines()`），其中每行都表示为一个JSONL对象的数据项。

2. 对文件中的每一行，将其转换为JSON对象（`json.loads()`）。

3. 提取图像的路径（`['imageGcsUri']`）和标签（`['classificationAnnotation']['displayName']`）。

In [ ]:
import json

import tensorflow as tf

with tf.io.gfile.GFile(jsonl_index, "r") as f:
    export_data_items = f.readlines()

for _ in range(10):
    j = json.loads(export_data_items[_])
    print(j["imageGcsUri"], j["classificationAnnotation"]["displayName"])

#### 创建TFRecords

接下来，我们需要创建一个数据提供机制，从数据集索引文件中向您将要训练的模型提供数据。有很多选择来构建一个数据提供机制。我们将在这里介绍两种选项，都使用TFRecords：

1. 将图像数据存储为原始未压缩的图像数据（每像素1字节）。
2. 将图像数据存储为经过预处理的数据 -- 机器学习准备好的 --（每像素4字节）。

这两种方法展示了磁盘存储和计算时间之间的权衡。在这两种情况下，我们都会对图像数据进行预处理，将数据缓存到一种形式中，以加速模型的训练时间。但是，通过缓存，我们既使用了磁盘空间，又增加了从磁盘到计算设备（例如CPU、GPU、TPU）的I/O流量。

在原始未压缩格式中，您最大限度地减少了磁盘上缓存数据的大小和I/O流量，但是在每个时代中，图像数据的预处理必须重复进行。在经过预处理的格式中，通过一次预处理并缓存预处理的数据，您最大限度地减少了计算时间 -- 即机器学习准备就绪。当以Float32进行训练时，磁盘上的数据量将增加四倍，同时您也将以相同数量增加磁盘空间和从磁盘到计算引擎的I/O流量。

辅助函数`TFExampleImageUncompressed`和`TFExampleImagePreprocessed`都接受以下参数：

- `path`：图像文件的Cloud Storage路径。
- `label`：图像文件对应的标签。
- `shape`：用于调整图像大小的（H，W）输入形状。如果`None`，则不进行调整大小。

辅助函数`TFExampleImagePreprocessed`具有一个额外参数：

- `dtype`：在像素数据归一化后的浮点表示。默认情况下，设置为32位浮点数（np.float32）。如果使用NVIDIA GPU或TPU，您可以通过设置`dtype = np.float16`来在16位浮点数下训练。在不影响准确性或时代数量的情况下，使用16位浮点数训练有两个好处：

1. 每次矩阵乘法运算比32位等效运算快4倍 -- 尽管模型权重也需要存储为16位。
2. 磁盘空间和I/O带宽减少了一半。

让我们更深入地了解创建`TFRecord`训练数据的函数。首先，`TFRecord`是训练数据的序列化二进制编码。作为一种编码，需要指定字段的编码方式的模式，然后在将训练数据提供给模型时用于解码。

模式被定义为在训练数据中每个数据项的`tf.train.Example`实例。每个`tf.train.Example`实例由一个序列字段组成，每个字段定义为键/值对。在我们的辅助函数中，键条目为：

- `image`：编码的原始图像数据。
- `label`：分配给图像的标签。
- `shape`：解码时图像的形状。

每个键/值对的值是`tf.train.Feature`的一个实例，其中：

- `bytes_list`：要编码的数据是一个字节串。
- `int64_list`：要编码的数据是一个或多个整数值的数组。
- `float_list`：要编码的数据是一个或多个浮点值的数组。

In [ ]:
import numpy as np


def TFExampleImageUncompressed(path, label, shape=None):
    """The uncompressed version of the image"""

    # read in (and uncompress) the image
    with tf.io.gfile.GFile(path, "rb") as f:
        data = f.read()
    image = tf.io.decode_image(data)

    if shape:
        image = tf.image.resize(image, shape)
    image = image.numpy()
    shape = image.shape

    # make the record
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image": tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image.tostring()])
                ),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                "shape": tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[shape[0], shape[1], shape[2]])
                ),
            }
        )
    )


def TFExampleImagePreprocessed(path, label, shape=None, dtype=np.float32):
    """The normalized version of the image"""

    # read in (uncompress) the image and normalize the pixel data
    image = (cv2.imread(path) / 255.0).astype(dtype)

    if shape:
        image = tf.image.resize(image, shape)
    image = image.numpy()
    shape = image.shape

    # make the record
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                "image": tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image.tostring()])
                ),
                "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                "shape": tf.train.Feature(
                    int64_list=tf.train.Int64List(value=[shape[0], shape[1], shape[2]])
                ),
            }
        )
    )

#### 将训练数据写入TFRecord文件

接下来，您将为在导出的数据集索引中指定的所有训练数据创建一个单个TFRecord：

- 通过将变量`CACHE`设置为`TFExampleImageUncompressed`或`TFExampleImagePreprocessed`来指定缓存方法。
- 使用`cls2label`将数据集中的类名转换为整数标签。
- 从导出的数据集索引文件（`tf.io.gfile.GFile(jsonl_index, 'r')`）中读取数据项列表。
- 将Cloud Storage位置设置为存储缓存TFRecord文件的位置-`GCS_TFRECORD_URI`。
- 使用`tf.io.TFRecordWriter(gcs_tfrecord_uri)`为导出的数据集索引中的每个数据项生成缓存数据。
- 提取Cloud Storage路径和类名-`json.loads(data_item)`。
- 将类名转换为整数标签-`label = cls2label[cls]`。
- 编码数据项-`example = CACHE(image, label)`。
- 将编码的数据项写入TFRecord文件-`writer.write(example.SerializeToString())`。

这可能需要大约20分钟。

In [ ]:
# Select TFRecord method of encoding
CACHE = TFExampleImageUncompressed  # [ TFExampleImageUncompressed, TFExampleImagePreprocessed]

# Map labels to class names
cls2label = {"daisy": 0, "dandelion": 1, "roses": 2, "sunflowers": 3, "tulips": 4}

# Read in each example from exported dataset index
with tf.io.gfile.GFile(jsonl_index, "r") as f:
    data = f.readlines()

# The path to the TFRecord cached file.
GCS_TFRECORD_URI = BUCKET_URI + "/flowers.tfrecord"

# Create the TFRecord cached file
with tf.io.TFRecordWriter(GCS_TFRECORD_URI) as writer:
    n = 0
    for data_item in data:
        j = json.loads(data_item)
        image = j["imageGcsUri"]
        cls = j["classificationAnnotation"]["displayName"]
        label = cls2label[cls]
        example = CACHE(image, label, shape=(128, 128))
        writer.write(example.SerializeToString())
        n += 1
        if n % 10 == 0:
            print(n, image)

listing = ! gsutil ls -la $GCS_TFRECORD_URI
print("TFRecord File", listing)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 存储桶

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
datasets = aip.TabularDataset.list(filter=f'display_name="example_{TIMESTAMP}"')
for dataset in datasets:
    dataset.delete()

# Delete the bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI